In [8]:
import re
from pathlib import Path
from typing import List, Union


def build_file_paths_regex(
    parent_dir: Union[str, Path],
    pattern: Union[str, List[str]],
    *,
    ignore_case: bool = False,
) -> List[str]:
    """
    Scan ``parent_dir`` with ``pathlib`` and return a list of *strings* that
    contain the absolute paths of files whose names match the supplied
    regular‑expression pattern(s).

    Parameters
    ----------
    parent_dir : str | pathlib.Path
        Directory that holds the target files.

    pattern : str | List[str]
        One regex pattern or a list of patterns applied to the filename
        (``Path.name``).  Example that matches the five ROOT files you listed::

            r"^rv02-02-01\\.sv02-02\\.mILD_l5_o2_v02\\.E250-SetA\\.I500078\\."
            r"P4f_zznu_sl\\.eL\\.pR\\.n000_\\d{3}\\.d_dst_00015656_\\d+\\.root$"

    ignore_case : bool, default=False
        If True, compile the regexes with ``re.IGNORECASE``.

    Returns
    -------
    List[str]
        Absolute path strings for each matching file, sorted alphabetically
        by filename.

    Example
    -------
    >>> regex = (
    ...     r"^rv02-02-01\\.sv02-02\\.mILD_l5_o2_v02\\.E250-SetA\\.I500078\\."
    ...     r"P4f_zznu_sl\\.eL\\.pR\\.n000_\\d{3}\\.d_dst_00015656_\\d+\\.root$"
    ... )
    >>> build_file_paths_regex("/my/data", regex)
    ['/my/data/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.'
     'P4f_zznu_sl.eL.pR.n000_001.d_dst_00015656_146.root',
     '/my/data/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.'
     'P4f_zznu_sl.eL.pR.n000_002.d_dst_00015656_70.root', …]
    """
    # --------------------------------------------------------------
    # Normalise the directory path
    parent_path = Path(parent_dir).expanduser().resolve()

    if not parent_path.is_dir():
        raise NotADirectoryError(f"'{parent_path}' is not a valid directory.")

    # --------------------------------------------------------------
    # Compile regex(es)
    if isinstance(pattern, str):
        patterns = [pattern]
    else:
        patterns = list(pattern)

    flags = re.IGNORECASE if ignore_case else 0
    compiled = [re.compile(p, flags) for p in patterns]

    # --------------------------------------------------------------
    # Collect matching files (non‑recursive – replace with rglob() for recursion)
    matched_strings: List[str] = []
    for entry in sorted(parent_path.iterdir()):   # deterministic order
        if not entry.is_file():
            continue

        if any(regex.search(entry.name) for regex in compiled):
            # Convert the Path object to its absolute string representation
            matched_strings.append(str(entry))

    return matched_strings


# ----------------------------------------------------------------------
# --------------------------- QUICK DEMO --------------------------------
if __name__ == "__main__":
    # Point this to the folder that actually contains your .root files
    parent = "../../input_files"

    # Regex that captures the five files you listed (and any similar ones)
    my_regex = (
        r"^rv02-02-01\.sv02-02\.mILD_l5_o2_v02\.E250-SetA\.I500078\."
        r"P4f_zznu_sl\.eL\.pR\.n000_\d{3}\.d_dst_00015656_\d+\.root$"
    )

    # Get a list of absolute path strings
    paths_as_strings = build_file_paths_regex(parent, my_regex)

    # Simple sanity‑check – print each path
    print(paths_as_strings)

['/home/mazeeqe/Documents/mestrado/code/tomaz-example/input_files/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.P4f_zznu_sl.eL.pR.n000_001.d_dst_00015656_146.root', '/home/mazeeqe/Documents/mestrado/code/tomaz-example/input_files/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.P4f_zznu_sl.eL.pR.n000_002.d_dst_00015656_70.root', '/home/mazeeqe/Documents/mestrado/code/tomaz-example/input_files/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.P4f_zznu_sl.eL.pR.n000_003.d_dst_00015656_201.root', '/home/mazeeqe/Documents/mestrado/code/tomaz-example/input_files/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.P4f_zznu_sl.eL.pR.n000_004.d_dst_00015656_167.root', '/home/mazeeqe/Documents/mestrado/code/tomaz-example/input_files/rv02-02-01.sv02-02.mILD_l5_o2_v02.E250-SetA.I500078.P4f_zznu_sl.eL.pR.n000_005.d_dst_00015656_91.root']
